In [2]:
import kagglehub
import pandas as pd
import re

In [3]:
# Download latest version
path = kagglehub.dataset_download("devdope/900k-spotify")

print("Path to dataset files:", path)

100%|██████████| 1.00G/1.00G [00:47<00:00, 22.6MB/s]

Extracting files...


Path to dataset files: C:\Users\zaxar\.cache\kagglehub\datasets\devdope\900k-spotify\versions\3


In [6]:
data = pd.read_csv(path + '\\spotify_dataset.csv')
data

,Artist(s),song,text,Length,emotion,Genre,Album,Release Date,Key,Tempo,...,Good for Morning Routine,Similar Artist 1,Similar Song 1,Similarity Score 1,Similar Artist 2,Similar Song 2,Similarity Score 2,Similar Artist 3,Similar Song 3,Similarity Score 3
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,sadness,hip hop,Thr!!!er,29th April 2013,D min,105,...,0,Corey Smith,If I Could Do It Again,0.986061,Toby Keith,Drinks After Work,0.983719,Space,Neighbourhood,0.983236
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,sadness,hip hop,Thr!!!er,29th April 2013,A# min,117,...,0,Hiroyuki Sawano,BRE@TH//LESS,0.995409,When In Rome,Heaven Knows,0.990905,Justice Crew,Everybody,0.984483
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,joy,hip hop,Louden Up Now,8th June 2004,A Maj,121,...,0,Ricky Dillard,More Abundantly Medley Live,0.993176,Juliet,Avalon,0.965147,The Jacksons,Lovely One,0.956752
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,joy,hip hop,As If,16th October 2015,A min,122,...,0,Eric Clapton,Man Overboard,0.992749,Roxette,Don't Believe In Accidents,0.991494,Tiwa Savage,My Darlin,0.990381
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,joy,hip hop,As If,16th October 2015,F min,123,...,0,Cibo Matto,Lint Of Love,0.981610,Barrington Levy,Better Than Gold,0.981524,Freestyle,Its Automatic,0.981415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551438,박혜진 Park Hye Jin,Can you,Can you be my (Can you be my) Can you be my (C...,03:39,joy,house,Can you (Galcher Lustwerk Remix),14th October 2020,B min,140,...,0,LXST CXNTURY,NEVER EXISTED,0.946429,"Occvlt,StoneDogg",ALIZAY,0.926155,Peaches,Light in Places,0.923749
551439,심형진 Hyungjin Sim,Lim Han Byul Lee Hyuk - Prayer English Transl...,[Verse] I gave you my everything But I see I c...,08:47,joy,metalcore,비 준비하시니 Psalm_147,19th April 2022,G# Maj,128,...,0,Kari Jobe,No Fear Live,0.991961,"The Belonging Co,Kari Jobe",Closer To Your Heart,0.990956,Saintseneca,Pillar of Na,0.990385
551440,우정하,Memories Smiling Tears Remix,"[Tasha] Hell on earth, watching ...",03:53,sadness,hip hop,MEMORY,30th August 2023,G# min,75,...,0,Manic Street Preachers,People Give In,0.982809,Vinyl Theatre,Gold,0.980835,Unspoken,Life in the Death of Me,0.980171
551441,제노,Come Back to Me,[Verse] Gifts and flowers lay upon the lid Of ...,03:12,sadness,hip hop,2008 to me again,3rd June 2008,E Maj,126,...,0,Bonnie Raitt,I Can't Help Myself,0.983805,J. Roddy Walston & The Business,Bad Habits,0.982425,Idina Menzel,Damsel In Distress,0.975935


In [7]:
# Let's only take audio features and raw text
# Audio freatures like key or tempo we can use in catboost model that will be used trained by us text feature extractor
data = data[['Artist(s)', 'song', 'text', 'Length', 'Album', 'Release Date', 'Key', 'Tempo', 'Loudness (db)', 'Time signature', 'Genre']]
data

,Artist(s),song,text,Length,Album,Release Date,Key,Tempo,Loudness (db),Time signature,Genre
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,Thr!!!er,29th April 2013,D min,105,-6.85db,4/4,hip hop
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,Thr!!!er,29th April 2013,A# min,117,-5.75db,4/4,hip hop
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,Louden Up Now,8th June 2004,A Maj,121,-6.06db,4/4,hip hop
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,As If,16th October 2015,A min,122,-5.42db,4/4,hip hop
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,As If,16th October 2015,F min,123,-5.57db,4/4,hip hop
...,...,...,...,...,...,...,...,...,...,...,...
551438,박혜진 Park Hye Jin,Can you,Can you be my (Can you be my) Can you be my (C...,03:39,Can you (Galcher Lustwerk Remix),14th October 2020,B min,140,-10db,4/4,house
551439,심형진 Hyungjin Sim,Lim Han Byul Lee Hyuk - Prayer English Transl...,[Verse] I gave you my everything But I see I c...,08:47,비 준비하시니 Psalm_147,19th April 2022,G# Maj,128,-9.68db,4/4,metalcore
551440,우정하,Memories Smiling Tears Remix,"[Tasha] Hell on earth, watching ...",03:53,MEMORY,30th August 2023,G# min,75,-4.67db,4/4,hip hop
551441,제노,Come Back to Me,[Verse] Gifts and flowers lay upon the lid Of ...,03:12,2008 to me again,3rd June 2008,E Maj,126,-3.93db,4/4,hip hop


In [8]:
data = data.rename(columns={'Artist(s)': 'artist', 'song': 'title', 'text': 'lyrics', 'Length': 'length', 'Album': 'album_title', 'Release Date': 'release', 'Key': 'key', 'Tempo': 'tempo', 'Loudness (db)': 'loudness', 'Time signature':'time_signature', 'Genre': 'genre'})
data

,artist,title,lyrics,length,album_title,release,key,tempo,loudness,time_signature,genre
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,Thr!!!er,29th April 2013,D min,105,-6.85db,4/4,hip hop
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,Thr!!!er,29th April 2013,A# min,117,-5.75db,4/4,hip hop
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,Louden Up Now,8th June 2004,A Maj,121,-6.06db,4/4,hip hop
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,As If,16th October 2015,A min,122,-5.42db,4/4,hip hop
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,As If,16th October 2015,F min,123,-5.57db,4/4,hip hop
...,...,...,...,...,...,...,...,...,...,...,...
551438,박혜진 Park Hye Jin,Can you,Can you be my (Can you be my) Can you be my (C...,03:39,Can you (Galcher Lustwerk Remix),14th October 2020,B min,140,-10db,4/4,house
551439,심형진 Hyungjin Sim,Lim Han Byul Lee Hyuk - Prayer English Transl...,[Verse] I gave you my everything But I see I c...,08:47,비 준비하시니 Psalm_147,19th April 2022,G# Maj,128,-9.68db,4/4,metalcore
551440,우정하,Memories Smiling Tears Remix,"[Tasha] Hell on earth, watching ...",03:53,MEMORY,30th August 2023,G# min,75,-4.67db,4/4,hip hop
551441,제노,Come Back to Me,[Verse] Gifts and flowers lay upon the lid Of ...,03:12,2008 to me again,3rd June 2008,E Maj,126,-3.93db,4/4,hip hop


In [9]:
data['lyrics'][551440]

"[Tasha]               Hell on earth, watching all lives pass by Nothing but the truth, it hurts, you can't deny [Sphere of Influence] Hell on earth No pausin' for love, with cats run up It's all about flossin', we stuck We out of luck [Tasha]               What, what, what [Sphere of Influence] Check it out [Tasha]               Lady T [Sphere of Influence] Sphere of Influence Sakura [Tasha]               Just a state of mind  [Chorus 1: Sakura] All the memories of hate And the lies Don't you know eventually we pay the price All my hopes and the dreams will survie Realize, we got to keep the faith alive  [Verse 1: Tasha] Damn How did we get here? We're living in fear How history repeats itself, can never steer clear Cries for help, ringing my ear from the sheer terror of it all I shed a tear, can't disappear, sometimes I wish the end was near But Back to reality, where Life still exist, we're Still on this earth, but we got shackles on the wrist Must be the reason why you never start 

In [10]:
# удаляем все табуляции, переносы и тд
data['lyrics'] = data['lyrics'].str.replace(r'[\t\n\r]+', ' ', regex=True)
data['lyrics'] = data['lyrics'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [28]:
data['lyrics'][551440]

"[Tasha] Hell on earth, watching all lives pass by Nothing but the truth, it hurts, you can't deny [Sphere of Influence] Hell on earth No pausin' for love, with cats run up It's all about flossin', we stuck We out of luck [Tasha] What, what, what [Sphere of Influence] Check it out [Tasha] Lady T [Sphere of Influence] Sphere of Influence Sakura [Tasha] Just a state of mind [Chorus 1: Sakura] All the memories of hate And the lies Don't you know eventually we pay the price All my hopes and the dreams will survie Realize, we got to keep the faith alive [Verse 1: Tasha] Damn How did we get here? We're living in fear How history repeats itself, can never steer clear Cries for help, ringing my ear from the sheer terror of it all I shed a tear, can't disappear, sometimes I wish the end was near But Back to reality, where Life still exist, we're Still on this earth, but we got shackles on the wrist Must be the reason why you never start to even lend a hand Too caught up in your ways for you to 

In [11]:
def only_ru_en_words(text):
    # Делим строку на слова (убираем пунктуацию)
    words = re.findall(r'\b\w+\b', text)
    for word in words:
        # Если слово не русское и не английское (то есть есть символы кроме a-z, A-Z, а-я, А-Я, ё, Ё, 0-9)
        if not re.fullmatch(r'[a-zA-Zа-яА-ЯёЁ0-9]+', word):
            return False  # есть "левое" слово → удаляем строку
    # Если есть хотя бы одно русское и хотя бы одно английское слово (или только один язык)
    return True

data_clean = data[data['lyrics'].apply(only_ru_en_words)]

In [12]:
data_clean

,artist,title,lyrics,length,album_title,release,key,tempo,loudness,time_signature,genre
0,!!!,Even When the Waters Cold,Friends told her she was better off at the bot...,03:47,Thr!!!er,29th April 2013,D min,105,-6.85db,4/4,hip hop
1,!!!,One Girl / One Boy,"Well I heard it, playing soft From a drunken b...",04:03,Thr!!!er,29th April 2013,A# min,117,-5.75db,4/4,hip hop
2,!!!,Pardon My Freedom,"Oh my god, did I just say that out loud? Shoul...",05:51,Louden Up Now,8th June 2004,A Maj,121,-6.06db,4/4,hip hop
3,!!!,Ooo,[Verse 1] Remember when I called you on the te...,03:44,As If,16th October 2015,A min,122,-5.42db,4/4,hip hop
4,!!!,Freedom 15,[Verse 1] Calling me like I got something to s...,06:00,As If,16th October 2015,F min,123,-5.57db,4/4,hip hop
...,...,...,...,...,...,...,...,...,...,...,...
551436,박진석,Jung Seung Hwan - Day Night English Translation,It was a gentle sound A light that shined in m...,03:42,Jinseok Park`s Trot Dictionary,1st April 2011,C min,136,-6.89db,4/4,hip hop
551439,심형진 Hyungjin Sim,Lim Han Byul Lee Hyuk - Prayer English Transl...,[Verse] I gave you my everything But I see I c...,08:47,비 준비하시니 Psalm_147,19th April 2022,G# Maj,128,-9.68db,4/4,metalcore
551440,우정하,Memories Smiling Tears Remix,"[Tasha] Hell on earth, watching all lives pass...",03:53,MEMORY,30th August 2023,G# min,75,-4.67db,4/4,hip hop
551441,제노,Come Back to Me,[Verse] Gifts and flowers lay upon the lid Of ...,03:12,2008 to me again,3rd June 2008,E Maj,126,-3.93db,4/4,hip hop


In [13]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534101 entries, 0 to 551442
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   artist          534101 non-null  object
 1   title           534086 non-null  object
 2   lyrics          534101 non-null  object
 3   length          534101 non-null  object
 4   album_title     534052 non-null  object
 5   release         534101 non-null  object
 6   key             534101 non-null  object
 7   tempo           534101 non-null  int64 
 8   loudness        534101 non-null  object
 9   time_signature  534093 non-null  object
 10  genre           534101 non-null  object
dtypes: int64(1), object(10)
memory usage: 48.9+ MB


In [14]:
# Очищаем от всех нестандартных символов
def clean_text(text):
    allowed = r'[^a-zA-Zа-яА-ЯёЁ0-9\s.,:;!?\'"\-\(\)\[\]\\/ $%#&]'
    return re.sub(allowed, '', text)

data_clean['lyrics'] = data_clean['lyrics'].apply(clean_text)
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534101 entries, 0 to 551442
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   artist          534101 non-null  object
 1   title           534086 non-null  object
 2   lyrics          534101 non-null  object
 3   length          534101 non-null  object
 4   album_title     534052 non-null  object
 5   release         534101 non-null  object
 6   key             534101 non-null  object
 7   tempo           534101 non-null  int64 
 8   loudness        534101 non-null  object
 9   time_signature  534093 non-null  object
 10  genre           534101 non-null  object
dtypes: int64(1), object(10)
memory usage: 48.9+ MB


C:\Users\zaxar\AppData\Local\Temp\ipykernel_7180\3696764116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['lyrics'] = data_clean['lyrics'].apply(clean_text)


In [15]:
# удаляем полные дубликаты по текстам
data_clean = data_clean.drop_duplicates(subset=['lyrics']).reset_index(drop=True)
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481756 entries, 0 to 481755
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   artist          481756 non-null  object
 1   title           481743 non-null  object
 2   lyrics          481756 non-null  object
 3   length          481756 non-null  object
 4   album_title     481710 non-null  object
 5   release         481756 non-null  object
 6   key             481756 non-null  object
 7   tempo           481756 non-null  int64 
 8   loudness        481756 non-null  object
 9   time_signature  481748 non-null  object
 10  genre           481756 non-null  object
dtypes: int64(1), object(10)
memory usage: 40.4+ MB


In [16]:
data_clean.to_csv('data/spotify_480k_with_features.csv', index=False)